This notebook is to run some exploratory work on the cloudquery database in a code-based (crucially, unit testable) way.

In [ ]:
import boto3
import psycopg2 as ps
import json
import sqlalchemy as sa
import pandas as pd

REGION="eu-west-1"

#gets the credentials from .aws/credentials
session = boto3.Session(profile_name='deployTools')

#get secret
secret_client = session.client('secretsmanager', REGION)
paginator = secret_client.get_paginator('list_secrets')

response_iterator = paginator.paginate(
    Filters=[
        {
            'Key': 'tag-value',
            'Values': [
                'cloud-query' #App
            ]
        }
    ]
)
secret_arn = list(response_iterator)[0]["SecretList"][0]['ARN']
secret = secret_client.get_secret_value(
    SecretId=secret_arn
)["SecretString"]

secret_json = json.loads(secret)

In [ ]:
import sqlalchemy as sa

import psycopg2 as ps
import pandas as pd
import boto3

conn = ps.connect(
    host=secret_json['host'],
    port=secret_json['port'],
    dbname='',
    user=secret_json['username'],
    password=secret_json['password'])

engine = sa.create_engine('postgresql://', creator=lambda: conn)

def select(table_name:str, columns: list[str], db_engine = engine) -> pd.DataFrame:
    return pd.read_sql_table(table_name, con=db_engine, columns=columns)

In [ ]:

topics_df = select("guardian_production_status", ['status', 'priority'])
topics_list = topics_df['status'].tolist()
print(topics_df['status'].tolist())

In [ ]:

non_pe_teams_list = select('guardian_non_p_and_e_github_teams', ['team_name'])['team_name'].tolist()
topics_df = select("github_repositories", ['full_name', 'topics'])
teams_df = select('github_teams', ['name', 'slug'])
#select function doesn't work on views, so we have to use read_sql_query
ownership_df = pd.read_sql_query("select repo_name, github_team_name, github_team_id from view_repo_ownership", con=conn)
new_df = ownership_df.merge(teams_df, how='left', left_on='github_team_name', right_on='name')[['repo_name', 'github_team_name', 'slug']]


In [ ]:
from rules import repository_06
topic_rule_df = repository_06(new_df, topics_df, topics_list, non_pe_teams_list)
freq = topic_rule_df['repository_06'].value_counts()
freq.plot.pie(subplots=True, figsize=(11, 6), title='Repo has an appropriate topic or is owned by a non-P&E team')

In [ ]:
repos_01_df = select('github_repositories',['full_name', 'archived', 'default_branch']) #this includes archived repos. make a decision about this

#create a new column that contains a boolean value for whether the default branch is called main
repos_01_df['repository_01'] = repos_01_df['default_branch'] == 'main'
repos_01_df =repos_01_df.drop(columns=['archived', 'default_branch'])
repos_01_df


In [ ]:
repository_rule_df =  repos_01_df.merge(topic_rule_df, how='left', left_on='full_name', right_on='repo_name')[['full_name', 'repository_01', 'repository_06']]
repository_rule_df.to_sql('repocop_github_repository_rules', engine, if_exists='replace', index=False)
repository_rule_df.head(10)

In [ ]:
pd.read_sql_query("select * from repocop_github_repository_rules", con=conn)

In [ ]:
freq = ownership_df['github_team_name'].value_counts()
freq

In [ ]:
import matplotlib.pyplot as plt
freq[:15].plot(kind='bar', title='Frequency Count of team name', xlabel='team name', ylabel='Count')
plt.show()

In [ ]:

topics_df = select("guardian_production_status", ['status', 'priority'])
topics_list = topics_df['status'].tolist()
print(topics_df['status'].tolist())

In [ ]:

non_pe_teams_list = select('guardian_non_p_and_e_github_teams', ['team_name'])['team_name'].tolist()
topics_df = select("github_repositories", ['full_name', 'topics'])
teams_df = select('github_teams', ['name', 'slug'])
#select function doesn't work on views, so we have to use read_sql_query
ownership_df = pd.read_sql_query("select repo_name, github_team_name, github_team_id from view_repo_ownership", con=conn)
new_df = ownership_df.merge(teams_df, how='left', left_on='github_team_name', right_on='name')[['repo_name', 'github_team_name', 'slug']]


In [ ]:
from rules import repository_06
topic_rule_df = repository_06(new_df, topics_df, topics_list, non_pe_teams_list)
freq = topic_rule_df['repository_06'].value_counts()
freq.plot.pie(subplots=True, figsize=(11, 6), title='Repo has an appropriate topic or is owned by a non-P&E team')

In [ ]:
freq = ownership_df['github_team_name'].value_counts()
freq

In [ ]:
import matplotlib.pyplot as plt
freq[:15].plot(kind='bar', title='Frequency Count of team name', xlabel='team name', ylabel='Count')
plt.show()